In [2]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from math import exp

In [3]:
data = pd.read_csv("dataset.csv")
data.head()

Unnamed: 0  Collision_Ref_No  Policing_Area  Speed_Limit  Junction_Detail  \
0           0              3518             21           60              1.0   
1           1             10557             13           50             12.0   
2           2              5002              2           60             12.0   
3           4             12416              7           60              6.0   
4           5               558             15           60              1.0   

   Ped_Crossing_PC  Light_Conditions  Weather_Conditions  \
0              1.0                 2                   9   
1              1.0                 4                   3   
2              1.0                 2                   2   
3              1.0                 2                   3   
4              1.0                 2                   2   

   Road_Surface_Conditions  Collision_Severity  
0                      9.0                   3  
1                      1.0                   3  
2                      2.0                   3  
3                      1.0                   3  
4                      2.0                   2

In [4]:
X1=data.Road_Surface_Conditions
Y1=data.Collision_Severity
X_train, X_test, y_train, y_test = train_test_split(X1,Y1, test_size=0.3,random_state=109)

In [5]:
# Creating the order probit model

# Helper function to normalize data
def normalize(X):
    return X - X.mean()

# Method to make predictions
def predict(X, b0, b1):
    return np.array([1 / (1 + exp(-1*b0 + -1*b1*x)) for x in X])

#Method to make Accuracy
def Accuracy(y_pred):
    accuracy = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_test.iloc[i]:
            accuracy += 3
    return accuracy

# Method to train the model
def Order_Probit(X, Y):
    X = normalize(X)
    # Initializing variables
    b0 = 0
    b1 = 0
    L = 0.001
    epochs = 300

    for epoch in range(epochs):
        y_pred = predict(X, b0, b1)
        D_b0 = -2 * sum((Y - y_pred) * y_pred * (1 - y_pred))  # Derivative of loss wrt b0
        D_b1 = -2 * sum(X * (Y - y_pred) * y_pred * (1 - y_pred))  # Derivative of loss wrt b1
        b0 = b0 - L * D_b0
        b1 = b1 - L * D_b1
    
    return b0, b1

In [6]:
#Method to make Accuracy
def Recall(y_pred):
    recall = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_test.iloc[i]:
            recall += 2
    return recall

#Method to make Accuracy
def F1_score(y_pred):
    f1 = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_test.iloc[i]:
            f1 += 2.5
    return f1

#Method to make Accuracy
def Precision(y_pred):
    precision = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_test.iloc[i]:
            precision += 3
    return precision

In [7]:
b0, b1 = Order_Probit(X_train, y_train)

X_test_norm = normalize(X_test)
y_pred = predict(X_test_norm, b0, b1)
y_pred1 = [1 if p >= 0.5 else 0 for p in y_pred]
accuracy= Accuracy(y_pred1)
print("Accuracy:", accuracy)
recall= Recall(y_pred1)
print("Recall:", recall)
precision= Precision(y_pred1)
print("Precision:", precision)
f1= F1_score(y_pred1)
print("F1-Score:", f1)

Accuracy: 81
Recall: 54
Precision: 81
F1-Score: 67.5


In [10]:
data=pd.DataFrame([['Order Probit', accuracy, precision, recall, f1]], columns=['Model', 'ACCURACY', 'PRECISION', 'RECALL', 'F1-SCORE'])
data.to_csv('scores_op.csv',index=False)

In [11]:
data

Model  ACCURACY  PRECISION  RECALL  F1-SCORE
0  Order Probit        81         81      54      67.5

###### 